Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [16]:
from random import random, choice, randint, sample, shuffle
from dataclasses import dataclass
from copy import copy
import numpy as np
import lab9_lib

In [17]:
NUM_GEN=1000
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15
INITIAL_ISLANDS=8
INITIAL_POPULATION=201
INITIAL_OFFSPRING=300

fitness = lab9_lib.make_problem(2)

In [18]:
@dataclass
class Individual:
    fitness: tuple
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, NUM_GEN-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def mutate_n(ind: Individual, n=2) -> Individual:
    offspring = copy(ind)
    pos = sample(range(NUM_GEN),n)
    for i in pos:
        offspring.genotype[i] = not offspring.genotype[i]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_GEN-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_GEN
    return offspring

def n_cut_xover(ind1: Individual, ind2: Individual, n = 4) -> Individual:
    cut_points = sample(range(1,NUM_GEN-1),n)
    cut_points.sort()
    genotype = []
    for i in range(n+1):
        if i == 0:
            genotype += ind1.genotype[:cut_points[i]]
        elif i!=n:
            if i % 2 == 0:
                genotype += ind1.genotype[cut_points[i-1]:cut_points[i]]
            else:
                genotype += ind2.genotype[cut_points[i-1]:cut_points[i]]
        else:
            if i % 2 == 0:
                genotype += ind1.genotype[cut_points[i-1]:]
            else:
                genotype += ind2.genotype[cut_points[i-1]:]

    offspring = Individual(fitness=None,
                           genotype= genotype)
    return offspring

def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    offspring = Individual(fitness=None,
                           genotype=[choice((g1, g2))
                                     for g1, g2 in zip(ind1.genotype, ind2.genotype)])
    return offspring

In [19]:
@dataclass
class Island_segregation:
    population: list[Individual]
    POPULATION_SIZE: int
    OFFSPRING_SIZE: int

    def make_initial_population(self) -> None:
        self.population = []
        self.population += [
        Individual(
            genotype=[choice((True, False)) for _ in range(NUM_GEN)],
            fitness=None,
        )
        for _ in range(self.POPULATION_SIZE//3)
        ]
        self.population += [
        Individual(
            genotype=[choice((True, False, False, False)) for _ in range(NUM_GEN)],
            fitness=None,
        )
        for _ in range(self.POPULATION_SIZE//3)
        ]
        self.population += [
        Individual(
            genotype=[choice((True,True, True, False)) for _ in range(NUM_GEN)],
            fitness=None,
        )
        for _ in range(self.POPULATION_SIZE//3)
        ]
        
        for i in self.population:
            i.fitness = fitness(i.genotype)
    
    def evolve_population(self, max_generation = 700) -> float:
        generation = 0
        while self.population[0].fitness < 1.0 and generation < max_generation:
            offspring = list()
            for _ in range(self.OFFSPRING_SIZE):
                if random() < MUTATION_PROBABILITY:  
                    p = select_parent(self.population)
                    o = mutate_n(p, n=2)
                else:
                    p1 = select_parent(self.population)
                    p2 = select_parent(self.population)
                    o = uniform_xover(p1, p2)
                offspring.append(o)

            for i in offspring:
                i.fitness = fitness(i.genotype)
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.POPULATION_SIZE]
            generation += 1
            
        return self.population[0].fitness


In [20]:
islands=[]
for i in range(INITIAL_ISLANDS):
    islands.append(Island_segregation(population=[],POPULATION_SIZE=INITIAL_POPULATION, OFFSPRING_SIZE=INITIAL_OFFSPRING))
    islands[i].make_initial_population()
number_of_fusions=1
while len(islands) > 0:
    for i in islands:
        best=i.evolve_population()
        print(f"Island {islands.index(i)} Fitness: {best}")
        if best == 1.0:
            print(f"Found with fitness call: {fitness.calls}")
            break
    
    if len(islands) >1:
        islands_tmp=[]
        number_of_fusions *= 2
        for i in range(0,len(islands),2):
            islands_tmp.append(Island_segregation(population=islands[i].population.deepcopy() + islands[i+1].population.deepcopy(), POPULATION_SIZE=INITIAL_POPULATION*number_of_fusions, OFFSPRING_SIZE=INITIAL_OFFSPRING*number_of_fusions))
        islands = islands_tmp
        print("---------------------------------------")

    else:
        print(f"Found: {best} with fitness call: {fitness.calls}")
        break

